# Two-dimensional test data generation
This notebook implements the data-generation procedure.
$$
\begin{aligned}
\partial_t c + \text{div}(-\alpha\mathbb{D}\nabla c) + r c &= 0 \\
\text{div}(-\alpha\mathbb{D}\nabla c) \cdot \vec{n} &= k(c - g(t))
\end{aligned}
$$

Where 
- $\alpha = 5$
- $\mathbb{D} = (2\times10^{-4}$ mm$^2$/s) $I$
- $r = 9.1\times10^{-6}$1/s
- $\hat k = 1.7\times10^{-3}$mm/s
- $g = \phi c_{\partial\Omega}$

In [1]:
import pint
import json

In [33]:
ureg = pint.get_application_registry()
mm = ureg.mm
s = ureg.s
parameters = dict(    
    a = 5,
    phi = 0.22,
    D_ = (2e-4 * mm**2 / s),
    r = (9.1e-6 * 1 / s),
    k = (1.7e-3 * mm / s),
)
coefficients = {
    "a": parameters["a"],
    "phi": parameters["phi"],
    "D_": parameters["D_"].to("mm^2/s").magnitude,
    "r": parameters["r"].to("1/s").magnitude,
    "k": parameters["k"].to("mm/s").magnitude,
}
print(json.dumps(coefficients, indent=2))

{
  "a": 5,
  "phi": 0.22,
  "D_": 0.0002,
  "r": 9.1e-06,
  "k": 0.0017
}


In [42]:
import numpy as np
import dolfin as df
import tqdm
import pantarei as pr
from dolfin import inner, grad, dot

from glymphopt.datageneration import BoundaryConcentration
from glymphopt.measure import measure
from glymphopt.parameters import parameters_2d_default
from glymphopt.timestepper import TimeStepper


def generate_data(domain_path, outputdir, tscale=1.0):
    acquisition_times = tscale * np.array([0.0, 4.81, 26.34, 48.32, 70.91])

    dt = 0.1 * tscale
    endtime = np.ceil(acquisition_times)[-1]

    timestepper = TimeStepper(dt, (0.0, endtime))
    timepoints = timestepper.vector()

    with df.HDF5File(df.MPI.comm_world, str(domain_path), "r") as hdf:
        domain = pr.read_domain(hdf)

    V = df.FunctionSpace(domain, "CG", 1)
    c_bdry = BoundaryConcentration(V, timescale=tscale)

    coefficients = parameters_2d_default()
    dt = timestepper.dt
    r = coefficients["r"]
    k = coefficients["k"]
    a = coefficients["a"]
    D_ = coefficients["D_"]
    aD = a * (D_ * df.Identity(2))  # type: ignore
    print(coefficients)

    u0 = df.Function(V)
    dx = df.Measure("dx", domain)
    ds = df.Measure("ds", domain)
    u, v = df.TrialFunction(V), df.TestFunction(V)

    a = inner(u, v) * dx + dt * (
        inner(dot(aD, grad(u)), grad(v)) * dx
        + r * inner(u, v) * dx  # type: ignore
        + k * inner(u, v) * ds  # type: ignore
    )
    L = u0 * v * dx + dt * k * (phi * c_bdry) * v * ds  # type: ignore

    assembler = df.SystemAssembler(a, L)
    A, b = df.PETScMatrix(), df.PETScVector()
    assembler.assemble(A, b)
    solver = df.LUSolver(A)

    Y = [df.Function(V, name="concentration") for _ in range(len(timepoints))]
    for n, tn in enumerate(tqdm.tqdm(timepoints[1:]), start=1):
        u0.assign(Y[n - 1])
        c_bdry.update(tn)
        assembler.assemble(b)
        solver.solve(Y[n].vector(), b)

    xdmf_boundary = df.XDMFFile(
        domain.mpi_comm(), f"{outputdir}/true_boundary_concentration.xdmf"
    )
    xdmf_internal = df.XDMFFile(
        domain.mpi_comm(), f"{outputdir}/true_concentration.xdmf"
    )
    for n, tn in enumerate(timepoints):
        xdmf_boundary.write(c_bdry.update(tn), t=tn)
        xdmf_internal.write(Y[n], t=tn)
    xdmf_boundary.close()
    xdmf_internal.close()

    print(timestepper, acquisition_times)
    Ym = measure(timestepper, Y, acquisition_times)

    xdmf_measured = df.XDMFFile(
        domain.mpi_comm(), f"{outputdir}/measured_concentration.xdmf"
    )
    hdf_measured = df.HDF5File(
        domain.mpi_comm(), f"{outputdir}/concentrations.hdf", "w"
    )
    for i, ti in enumerate(acquisition_times):
        xdmf_measured.write(Ym[i], t=ti)
        if i == 0:
            pr.write_function(hdf_measured, Ym[i], "concentration")
            pr.write_function(hdf_measured, c_bdry.update(ti), "boundary_concentration")
        else:
            pr.write_checkpoint(hdf_measured, Ym[i], "concentration", t=ti)
            pr.write_checkpoint(
                hdf_measured, c_bdry.update(ti), "boundary_concentration", t=ti
            )

    xdmf_measured.close()
    hdf_measured.close()


if __name__ == "__main__":
    generate_data("../resources/brain-2d-domain.hdf", "../resources", tscale=3600)


{'a': 5, 'phi': 0.22, 'D_': 0.0002, 'r': 9.1e-06, 'k': 0.0017}
Calling FFC just-in-time (JIT) compiler, this may take some time.


100%|████████████████████████████████████████| 710/710 [00:01<00:00, 420.25it/s]


TimeStepper(dt=360.0, interval=(0.0, np.float64(255276.0))) [     0.  17316.  94824. 173952. 255276.]
